In [2]:
%pip install pandas
%pip install ace_tools

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install math

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import glob

# 각도 계산 함수
def calculate_angle(point_a, point_b, point_c):
    vector_ab = np.array(point_a) - np.array(point_b)
    vector_cb = np.array(point_c) - np.array(point_b)
    dot_product = np.dot(vector_ab, vector_cb)
    magnitude_ab = np.linalg.norm(vector_ab)
    magnitude_cb = np.linalg.norm(vector_cb)
    if magnitude_ab == 0 or magnitude_cb == 0:
        return None
    cosine_angle = dot_product / (magnitude_ab * magnitude_cb)
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

# 여러 CSV 파일 읽기
file_pattern = "push_csv_data/push_*.csv"  # 예: push_001.csv, push_002.csv, ...
files = glob.glob(file_pattern)

# 데이터 병합
all_data = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)

# 기준 각도 계산 (전체 데이터의 평균 및 범위 설정)
elbow_mean_angle = all_data['elbow_angle'].mean() if 'elbow_angle' in all_data else 90
hip_mean_angle = all_data['hip_angle'].mean() if 'hip_angle' in all_data else 170
ELBOW_MIN, ELBOW_MAX = elbow_mean_angle - 15, elbow_mean_angle + 15
HIP_MIN, HIP_MAX = hip_mean_angle - 10, hip_mean_angle + 10

# 겨드랑이 각도 기준 설정 (예제 기준)
ARMPIT_MIN, ARMPIT_MAX = 40, 50  # 겨드랑이 각도 범위 설정

print(f"팔꿈치 기준 각도: {ELBOW_MIN:.2f}° ~ {ELBOW_MAX:.2f}°")
print(f"엉덩이 기준 각도: {HIP_MIN:.2f}° ~ {HIP_MAX:.2f}°")
print(f"겨드랑이 기준 각도: {ARMPIT_MIN:.2f}° ~ {ARMPIT_MAX:.2f}°")

# Mediapipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# 웹캠 열기
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("웹캠을 열 수 없습니다.")
        break
    
    # Mediapipe로 관절 추출
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # 필요한 관절 좌표 추출
        def get_point(landmark):
            return [landmarks[landmark].x, landmarks[landmark].y, landmarks[landmark].z]
        
        try:
            shoulder = get_point(mp_pose.PoseLandmark.LEFT_SHOULDER)
            elbow = get_point(mp_pose.PoseLandmark.LEFT_ELBOW)
            wrist = get_point(mp_pose.PoseLandmark.LEFT_WRIST)
            hip = get_point(mp_pose.PoseLandmark.LEFT_HIP)
            knee = get_point(mp_pose.PoseLandmark.LEFT_KNEE)
            
            # 각도 계산
            elbow_angle = calculate_angle(shoulder, elbow, wrist)
            hip_angle = calculate_angle(shoulder, hip, knee)
            armpit_angle = calculate_angle(elbow, shoulder, hip)  # 겨드랑이 각도 계산
            
            # 잘못된 자세 확인
            feedback = []
            if not (ELBOW_MIN <= elbow_angle <= ELBOW_MAX):
                feedback.append("팔꿈치 각도 잘못됨")
            if not (HIP_MIN <= hip_angle <= HIP_MAX):
                feedback.append("엉덩이 각도 잘못됨")
            if not (ARMPIT_MIN <= armpit_angle <= ARMPIT_MAX):
                feedback.append("겨드랑이 각도 잘못됨")
            
            # 피드백 텍스트
            feedback_text = ", ".join(feedback) if feedback else "올바른 자세"
            
            # 텍스트 및 각도 표시
            cv2.putText(image, f"Elbow: {elbow_angle:.2f}°", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if not feedback else (0, 0, 255), 2)
            cv2.putText(image, f"Hip: {hip_angle:.2f}°", (50, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if not feedback else (0, 0, 255), 2)
            cv2.putText(image, f"Armpit: {armpit_angle:.2f}°", (50, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if not feedback else (0, 0, 255), 2)
            cv2.putText(image, feedback_text, (50, 140),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0) if not feedback else (0, 0, 255), 2)
        
        except Exception as e:
            print(f"오류 발생: {e}")
    
    # Mediapipe 시각화
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    # 이미지 출력
    cv2.imshow("Push-up Analysis", image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


팔꿈치 기준 각도: 75.00° ~ 105.00°
엉덩이 기준 각도: 160.00° ~ 180.00°
겨드랑이 기준 각도: 40.00° ~ 50.00°


I0000 00:00:1737518990.353579 8764578 gl_context.cc:369] GL version: 2.1 (2.1 ATI-6.1.13), renderer: AMD Radeon Pro 555X OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1737518990.653252 8794281 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737518990.693357 8794281 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737518991.658224 8794280 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2025-01-22 13:09:52.418 Python[98755:8764578] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-22 13:09:52.418 Python[98755:8764578] +[IMKInputSession subclass]: chose IMKInputSession_Modern
